In [1]:
##### Pre-processing with subject loops

import glob
import os

import mne
from mne.preprocessing import ICA

# Define montage used in the experiment
chanlocs = mne.channels.read_montage(kind='biosemi64')

In [ ]:
%%capture
# Set up subject loop to perform the following steps
# for pre-processing: band-pass filtering, re-referencing,
# and ICA

path = './rawdata'
for file in glob.glob(os.path.join(path, '*.bdf')):
    
    # Note the file details
    filepath, filename = os.path.split(file)
    filename, ext = os.path.splitext(filename)
    
    # Load the raw data
    raw = mne.io.read_raw_edf(file, montage=chanlocs, preload=True, stim_channel=-1,
                              eog=[u'EXG1', u'EXG2'], 
                              exclude=[u'EXG3', u'EXG4', u'EXG5', u'EXG6', u'EXG7', u'EXG8']) 
    picks = mne.pick_types(raw.info, eeg=True, eog=False)
    
    # Filter and re-reference the data
    raw.filter(0.1, 50) 
    raw.set_eeg_reference(ref_channels='average', projection=False) 
    
    # Save the results
    raw.save('./raw/' + filename + '-raw.fif', overwrite=True)
    
    # Specify ICA settings
    n_components = 25 
    method = 'extended-infomax'
    reject = dict(eeg=200e-6)
    
    # Perform ICA and save the results
    ica = ICA(n_components=n_components, method=method)
    ica.fit(raw.copy().filter(1,50), picks=picks, reject=None) 
    ica.save('./ica/' + filename + '-ica.fif')

In [ ]:
%%capture
# Load each subject's ICA solution and raw data file
ica = mne.preprocessing.read_ica('./ica/*-ica.fif')
raw = mne.io.read_raw_fif('./raw/*-raw.fif', preload=True)

# Investigate the components and pick the ones
# representing artifacts
ica.plot_components()
ica.plot_properties(raw, picks=[])
ica.apply(raw, exclude=[0,1,2,4,6,10,11,13,17,18,19,20,21,22,23])
raw.save('./raw/*_ICA-raw.fif', overwrite=True)